In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import copy
%matplotlib inline

In [2]:
import locale

print(locale.getpreferredencoding())

UTF-8


In [297]:
class Koma:
    def __init__(self,mode0Move,mode1Move,komaType):
        self.mode0Move = mode0Move
        self.mode1Move = mode1Move
        self.mode = 0
        self.directions = self.mode0Move
        self.type = komaType
    
    #コマの種類の取り出し
    def GetType(self):
        return self.type
    def Translate(self,lst):
        newLst = []
        for l in lst:
            if l < 0:
                l = abs(l)%9
                l = -l
            else:
                l = l%9
            newLst.append(l)
        return newLst
    #進行可能かのチェック
    def CheckDir(self,direction,player):
        #動きとしてできるかの検証
        x = 0
        y = 0
        if direction[0] > 0:
            x = 0
        elif direction[0]==0:
            x = 1
        else:
            x = 2
        if direction[1] > 0:
            y = 2
        elif direction[1] == 0:
            y = 1
        else:
            y = 0
        x = abs(x-player)
        y = abs(y-player)
        if self.directions[x][y] > 0:
            return True
        return False
    
    #移動可能方向の取り出し
    def GetMove(self):
        return self.directions
        
    #移動処理
    def Move(self,position,direction,player):
        Path = [position]
        judge1 = self.CheckDir(direction,player)
        if judge1:
            Path = self.MakePath(Path,position,direction,player,x,y)
        else:
            print("その動きはできません")
        return Path
    def GetMovableList(self,position,player,Board):
        movableList = self.MovableList(position,player)
        movableList = self.ReflectBoard(movableList,Board,position)
        return movableList
    #進行可能な方向へのリストを格納する
    def MovableList(self,position,player):
        movableList =[]
        #全方向ベクトルに対して
        for i in range(3):
            for j in range(3):
                if not i==1 or not j==1:
                    direction = self.GetDirections(i,j)
                    if not direction[0]==0 or not direction[1]==0:
                        #print(direction)
                        movableList.append(self.MakePath([],position,direction,player))
        movableList = list(map(list,set(map(tuple,movableList))))
        movableList = [x for x in movableList if x]
        return movableList
    
    #進行可能方向へのベクトルを獲得する
    def GetDirections(self,i,j):
        direction = self.directions[i][j]
        if direction == 0:
            return (0,0)
        else:
            x = 0
            y = 0
            if i==0:
                x = direction
            elif i==1:
                x = 0
            else:
                x = -direction
            if j==0:
                y = -direction
            elif j==1:
                y = 0
            else:
                y = direction
            return (x,y)   
    def MakePath(self,Path,position,direction,player):
        playerCorrection = 1-player
        #positionやdirectionが9以上の場合、8以内の値にする
        position = self.Translate(position)
        #print(direction)
        direction = self.Translate(direction)
        #x座標とy座標のスタート地点の設定
        x_start = position[0]
        y_start = position[1]
        #x座標とy座標のゴール地点の設定
        x_end = playerCorrection*direction[0]+position[0]
        y_end = playerCorrection*direction[1]+position[1]
        if x_start > x_end:
            x_end -=1
        else:
            x_end +=1
        if y_start > y_end:
            y_end -=1
        else:
            y_end +=1
        x_end,y_end = self.SetEnd(x_start,x_end,y_start,y_end)
        #print(x_end)
        #print(y_end)
        #開始店が盤面上でなかった場合、エラー出力
        if x_start < 0 or x_start > 8:
            print("指定された座標は盤面ガイです")
            return Path
        if y_start < 0 or y_start > 8:
            print("指定された座標は盤面ガイです")
            return Path
        #
        step_x = 1
        step_y = 1
        if x_start > x_end:
            step_x = -1
        if y_start > y_end:
            step_y = -1
        #Pathの作成
        prev_x = [10]
        prev_y = [10]
        x_end = int(x_end)
        y_end = int(y_end)
        for x in range(x_start,x_end,step_x):
            for y in range(y_start,y_end,step_y):
                judge_x = False
                judge_y = False
                #x
                if direction[0]==0:
                    if x-position[0]==0:
                        judge_x = True
                else:
                    if x-position[0]!=0:
                        if not x in prev_x:
                            judge_x = True
                #y        
                if direction[1]==0:
                    if y-position[1]==0:
                        judge_y = True
                else:
                    if y-position[1]!=0:
                        if not y in prev_y:
                            judge_y = True
                if judge_x and judge_y:
                    prev_x.append(x)
                    prev_y.append(y)
                    Path.append((x,y))
        direction_x = direction[0]
        direction_y = direction[1]
        if abs(direction_x) >0:
            direction_x = direction[0]/abs(direction[0])
        if abs(direction_y)>0:
            direction_y = direction[1]/abs(direction[1])
        return Path
    
    def SetEnd(self,x_start,x_end,y_start,y_end):
        #n_x,n_y:それぞれの軸への方向の移動回数
        n_x = 0
        n_y = 0
        #:flag_x,flag_y:盤面のはじにぶつかったか
        flag_x = False
        flag_y = False
        #dixt_x,dict_y:それぞれの軸への進行方向
        dict_x = 1
        dict_y = 1
        if x_start > x_end:
            dict_x = -1
        if y_start > y_end:
            dict_y = -1
        #x座標におけるオーバーフローがないかの確認
        for i in range(x_start,x_end,dict_x):
            if i < 0 or i > 8:
                flag_x = True
                break
            n_x += 1
        #y座標におけるオーバーフローがないかの確認
        for i in range(y_start,y_end,dict_y):
            if i < 0 or i > 8:
                flag_y = True
                break
            n_y += 1
        #オバーフローが発生しているならより早く発生している方に合わせる
        if  flag_x or flag_y:
            if flag_x and not flag_y:
                #x座標,y座標におけるendを設定する
                x_end = dict_x*n_x+x_start
                y_end = dict_y*n_x+y_start
            elif not flag_x and flag_y:
                #x座標,y座標におけるendを設定する
                x_end = dict_x*n_y+x_start
                y_end = dict_y*n_y+y_start
            elif flag_x and flag_y:
                if n_x > n_y:
                    #x座標,y座標におけるendを設定する
                    x_end = dict_x*n_y+x_start
                    y_end = dict_y*n_y+y_start
                else:
                    #x座標,y座標におけるendを設定する
                    x_end = dict_x*n_x+x_start
                    y_end = dict_y*n_x+y_start
        return x_end,y_end
    def ReflectBoard(self,movableList,Board,position):
        newList = []
        PlayerType = self.GetPlayerType(Board,position)
        #print(PlayerType)
        for path in movableList:
            newPath = []
            #print("path is "+str(path))
            IsProceed = True
            for node in path:
                if IsProceed:
                    if int(Board[node[0],node[1]]) ==0:
                        newPath.append(node)
                    elif not PlayerType == self.GetPlayerType(Board,node):
                        newPath.append(node)
                        IsProceed = False
                    else:
                        IsProceed = False 
                else:
                    break
            if newPath:
                newList.append(newPath)
        return newList
    def GetPlayerType(self,Board,position):
        if int(Board[position[0],position[1]]) ==0:
            return 0
        elif int(Board[position[0],position[1]]) %11==0:
            return 11
        elif int(Board[position[0],position[1]]) %13==0:
            return 13
        else:
            print(Board[position[0],position[1]])
            print("上記のコマ情報は存在しません")
            return 0
    #コマを反転させる
    def ChangeMove(self):
        self.mode = 1
        self.directions = self.mode1Move

In [298]:
class Fu(Koma):
    def __init__(self):
        self.mode0Move = [[0,1,0],[0,0,0],[0,0,0]]
        self.mode1Move = [[1,1,1],[1,0,1],[0,1,0]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Fu")
class Hisha(Koma):
    def __init__(self):
        self.mode0Move = [[0,8,0],[8,0,8],[0,8,0]]
        self.mode1Move = [[1,8,1],[8,0,8],[0,8,0]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Hisha")
        
class Kaku(Koma):
    def __init__(self):
        self.mode0Move = [[8,0,8],[0,0,0],[8,0,8]]
        self.mode1Move = [[8,1,8],[1,0,1],[8,1,8]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Hisha")
class Gyoku(Koma):
    def __init__(self):
        self.mode0Move = [[1,1,1],[1,0,1],[1,1,1]]
        self.mode1Move = [[1,1,1],[1,0,1],[1,1,1]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Gyoku")
        
class Gin(Koma):
    def __init__(self):
        self.mode0Move = [[1,1,1],[0,0,0],[1,0,1]]
        self.mode1Move = [[1,1,1],[1,0,1],[1,1,1]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Gin")
class Kin(Koma):
    def __init__(self):
        self.mode0Move = [[1,1,1],[1,0,1],[0,1,0]]
        self.mode1Move = [[1,1,1],[1,0,1],[0,1,0]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Kin")
        
class Keima(Koma):
    def __init__(self):
        self.mode0Move = [[2,0,2],[0,0,0],[0,0,0]]
        self.mode1Move = [[1,1,1],[1,0,1],[0,1,0]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Keima")
        
class Kyosha(Koma):
    def __init__(self):
        self.mode0Move = [[0,8,0],[0,0,0],[0,0,0]]
        self.mode1Move = [[1,1,1],[1,0,1],[0,1,0]]
        super().__init__(self.mode0Move,self.mode1Move,komaType="Kyosha")
        

In [299]:
class Game:
    def __init__(self):
        self.Komas = {"Fu":1,"Kyosha":2,"Keima":3,"Hisha":4,"Kaku":5,"Gin":6,"Kin":7,"Gyoku":8}
        self.Board = np.zeros((9,9))
        self.SetBoard()
        self.KomaMove = {1:Fu(),2:Kyosha(),3:Keima(),4:Hisha(),5:Kaku(),6:Gin(),7:Kin(),8:Gyoku()}
        self.Direction = {"UpLeft":0,"Up":1,"UpRight":2,"Left":3,"Right":4,"DownLeft":5,"Down":6,"DownRight":7}
    def SetBoard(self):
        #歩
        for i in range(9):
            self.Board[2,i] = self.Komas["Fu"]*11
            self.Board[6,i] = self.Komas["Fu"]*13
        #飛車
        self.Board[1,7] = self.Komas["Hisha"]*11
        self.Board[7,1] = self.Komas["Hisha"]*13
        
        #角
        self.Board[1,1] = self.Komas["Kaku"]*11
        self.Board[7,7] = self.Komas["Kaku"]*13
        
        #香車
        self.Board[0,0] = self.Komas["Kyosha"]*11
        self.Board[0,8] = self.Komas["Kyosha"]*11
        self.Board[8,0] = self.Komas["Kyosha"]*13
        self.Board[8,8] = self.Komas["Kyosha"]*13
        
        #桂馬
        self.Board[0,1] = self.Komas["Keima"]*11
        self.Board[0,7] = self.Komas["Keima"]*11
        self.Board[8,1] = self.Komas["Keima"]*13
        self.Board[8,7] = self.Komas["Keima"]*13
        
        #銀
        self.Board[0,2] = self.Komas["Gin"]*11
        self.Board[0,6] = self.Komas["Gin"]*11
        self.Board[8,2] = self.Komas["Gin"]*13
        self.Board[8,6] = self.Komas["Gin"]*13
        
        #金
        self.Board[0,3] = self.Komas["Kin"]*11
        self.Board[0,5] = self.Komas["Kin"]*11
        self.Board[8,3] = self.Komas["Kin"]*13
        self.Board[8,5] = self.Komas["Kin"]*13
        
        #王
        self.Board[0,4] = self.Komas["Gyoku"]*11
        self.Board[8,4] = self.Komas["Gyoku"]*13
        #初期配置変形
        self.Board[2,4] = 0
    def Start(self):
        pass
    def ChangeKomaBoard(self):
        Board = np.zeros((9,9))
        for j in range(9):
            for i in range(9):
                element = self.Board[i,j]
                if element == 0:
                    Board[i,j] = 0
                elif element % 11==0:
                    Board[i,j] = element/11
                elif element % 13==0:
                    Board[i,j] = element/13
                else:
                    print("存在しないはずのコマ情報が載っています")
        return Board
    def GetBoard(self):
        return self.Board
    def PrintBoard(self):
        Board = self.ChangeKomaBoard()
        print(Board)
    def GetKomaType(self):
        pass
    def CheckBoard(self,Path):
        n = 0
        Obstacle = 0
        pre_Obstacle  = 0 
        for pos in Path:
            if n>0 and self.Board[pos[0],pos[1]]!=0:
                Obstacle+=1
                if pre_Obstacle==0:
                    pre_Obstacle = n
                if n != pre_Obstacle:
                    print("障害物があります")
                    return True
            n+=1
        print("障害物はありません")
        return False
    
    def GetKomaMove(self):
        return self.KomaMove
    def BoardMove(self,position,direction,player):
        #positionのコマ情報を取得
        Koma = self.KomaMove[self.Board[position]]
        #コマとして動くことができるかをチェックし移動通路を返す
        Path = Koma.Move(position,direction,player,self.Board)
        #移動中に障害物があるかをチェックする。
        CheckObstacle = self.CheckBoard(Path)
        #コマをとる処理を行う
        if not CheckObstacle:
            self.Board[newPosition] = self.Board[position]
        if newPosition!=position:
            self.Board[position]=0

In [300]:
Shogi = Game()
Board = Shogi.GetBoard()
Shogi.PrintBoard()

[[2. 3. 6. 7. 8. 7. 6. 3. 2.]
 [0. 5. 0. 0. 0. 0. 0. 4. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 4. 0. 0. 0. 0. 0. 5. 0.]
 [2. 3. 6. 7. 8. 7. 6. 3. 2.]]


In [301]:
Path = []
position = (2,0)
direction = (6,8)
KomaMove=Shogi.GetKomaMove()
Path = KomaMove[1].MakePath(Path,position,direction,0)

In [302]:
Shogi.PrintBoard()

[[2. 3. 6. 7. 8. 7. 6. 3. 2.]
 [0. 5. 0. 0. 0. 0. 0. 4. 0.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 4. 0. 0. 0. 0. 0. 5. 0.]
 [2. 3. 6. 7. 8. 7. 6. 3. 2.]]


In [303]:
movableList = KomaMove[1].MovableList(position,0)

In [304]:
print(movableList)

[[(3, 0)]]


In [305]:
move = movableList[0]
Shogi.CheckBoard(move)

障害物はありません


False

In [306]:
print(move)

[(3, 0)]


In [307]:
XY_test = [(-1,9),(9,-1),(10,-1),(-1,10),(10,3),(3,10),(-1,-1),(10,10),(3,3)]
direction_test = [(0,0),(1,0),(-1,0),(0,1),(0,-1)]
assist = 1

In [308]:
Kyosha = KomaMove[4].MovableList((0,4),2)
print(Kyosha)

[[(1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4)], [(0, 5), (0, 6), (0, 7), (0, 8)], [(0, 3), (0, 2), (0, 1), (0, 0)]]


In [309]:
FinalList = KomaMove[4].ReflectBoard(Kyosha,Board,(0,4))

In [310]:
print(FinalList)

[[(1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]]


Shogi.PrintBoard()

In [311]:
KomaMove[4].GetMovableList((0,4),2,Board)

[[(1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]]